In [225]:
pip install torch_geometric

Note: you may need to restart the kernel to use updated packages.


In [226]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import collections
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.transforms import RandomNodeSplit as masking
from torch_geometric.utils.convert import to_networkx
from torch_geometric.nn import GCNConv

import networkx as nx

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv
/kaggle/input/ue20cs344-nam-assignment-2/sample_submission.csv
/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv
/kaggle/input/ue20cs344-nam-assignment-2/features.pt


In [227]:

nodes = pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/node_data.csv')
nodes

,id,branch,train_mask,test_mask
0,0,ECE,True,False
1,1,ECE,True,False
2,2,CSE,True,False
3,3,EEE,True,False
4,4,NaN,False,True
...,...,...,...,...
19712,19712,EEE,True,False
19713,19713,CSE,True,False
19714,19714,EEE,True,False
19715,19715,NaN,False,True


In [228]:
nodes['branch'] = nodes['branch'].replace(np.nan, 4)
nodes

,id,branch,train_mask,test_mask
0,0,ECE,True,False
1,1,ECE,True,False
2,2,CSE,True,False
3,3,EEE,True,False
4,4,4,False,True
...,...,...,...,...
19712,19712,EEE,True,False
19713,19713,CSE,True,False
19714,19714,EEE,True,False
19715,19715,4,False,True


In [229]:
nodes['branch'] = pd.factorize(nodes['branch'])[0].astype(np.uint16)
labels = nodes['branch']
labels.head()


0    0
1    0
2    1
3    2
4    3
Name: branch, dtype: uint16

In [230]:
edges = pd.read_csv('/kaggle/input/ue20cs344-nam-assignment-2/edgelist.csv')
edges

,source,target
0,0,1378
1,0,1544
2,0,6092
3,0,7636
4,0,14442
...,...,...
88643,19712,2361
88644,19713,12204
88645,19714,12278
88646,19715,4284


In [231]:
edge_list = list(zip(edges['source'], edges['target']))
g.add_edges_from(edge_list)
len(edge_list)

88648

In [232]:
list(g.edges(data=True))[0:5]

[(0, 1378, {}), (0, 1544, {}), (0, 6092, {}), (0, 7636, {}), (0, 14442, {})]

In [233]:
for idx, row in nodes.iterrows():
    g.add_node(idx, **row.to_dict())

In [234]:
list(g.nodes(data=True))[0:10]

[(0, {'id': 0, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (1378, {'id': 1378, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (1544, {'id': 1544, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (6092, {'id': 6092, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (7636, {'id': 7636, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (14442, {'id': 14442, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (1, {'id': 1, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (2943, {'id': 2943, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (8359, {'id': 8359, 'branch': 0, 'train_mask': True, 'test_mask': False}),
 (10199, {'id': 10199, 'branch': 3, 'train_mask': False, 'test_mask': True})]

In [235]:
print('# of edges: {}'.format(g.number_of_edges()))
print('# of nodes: {}'.format(g.number_of_nodes()))

# of edges: 44324
# of nodes: 19717


In [236]:
import itertools
deg_centrality = nx.degree_centrality(g)
out = dict(itertools.islice(deg_centrality.items(), 10))
print(out)

{0: 0.0002536011361330899, 1378: 0.0018259281801582471, 1544: 0.0017752079529316292, 6092: 0.0012172854534388314, 7636: 0.0010651247717589774, 14442: 0.0010144045445323595, 1: 0.00015216068167985393, 2943: 0.0006086427267194157, 8359: 0.0009636843173057416, 10199: 0.0002028809089064719}


In [237]:
from torch_geometric.utils.convert import from_networkx

In [238]:
data = from_networkx(g)

In [239]:
y = torch.tensor(labels , dtype=torch.long)
data.y=y

In [240]:
data

Data(edge_index=[2, 88648], id=[19717], branch=[19717], train_mask=[19717], test_mask=[19717], num_nodes=19717, y=[19717])

In [241]:
features = torch.load('/kaggle/input/ue20cs344-nam-assignment-2/features.pt')
data.x=features

In [242]:
data

Data(edge_index=[2, 88648], id=[19717], branch=[19717], train_mask=[19717], test_mask=[19717], num_nodes=19717, y=[19717], x=[19717, 500])

In [243]:
data.y[:10]

tensor([0, 0, 1, 2, 3, 2, 2, 0, 2, 0])

In [244]:
data.test_mask = torch.tensor(nodes.test_mask.tolist())
data.train_mask = torch.tensor(nodes.train_mask.tolist())

#### MODEL

In [245]:
import torch
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv 

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()

        # Initialize the layers
        self.conv1 = GCNConv(500, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.out = Linear(hidden_channels, 3)

    def forward(self, x, edge_index):
        # First Message Passing Layer (Transformation)
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        # Second Message Passing Layer
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        
        #3rd
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)


        # Output layer 
        x = F.softmax(self.out(x), dim=1)
        return x

model = GCN(hidden_channels=32)
print(model)

GCN(
  (conv1): GCNConv(500, 32)
  (conv2): GCNConv(32, 32)
  (conv3): GCNConv(32, 32)
  (out): Linear(in_features=32, out_features=3, bias=True)
)


In [251]:
learning_rate = 0.05
decay = 5e-4
optimizer = torch.optim.Adam(model.parameters(), 
                             lr=learning_rate, 
                             weight_decay=decay)

criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad() 
      # Use all data as input, because all nodes have node features
      out = model(data.x, data.edge_index)  
      # Only use nodes with labels available for loss calculation --> mask
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  
      loss.backward() 
      optimizer.step()
      return loss
    
losses = []
for epoch in range(0, 500):
    loss = train()
    losses.append(loss)
    if epoch % 100 == 0:
      print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')


Epoch: 000, Loss: 1.0642
Epoch: 100, Loss: 1.0642
Epoch: 200, Loss: 1.0641
Epoch: 300, Loss: 1.0642
Epoch: 400, Loss: 1.0641


In [247]:
pred = model(data.x, data.edge_index)
pred

tensor([[0.4959, 0.0074, 0.4967],
        [0.5071, 0.0150, 0.4779],
        [0.4806, 0.0250, 0.4944],
        ...,
        [0.5000, 0.0072, 0.4928],
        [0.4863, 0.0026, 0.5111],
        [0.4671, 0.0066, 0.5263]], grad_fn=<SoftmaxBackward0>)

In [248]:
my_pred = torch.argmax(pred, dim=1)
node_ids = []
for i in range(0, 19717):
    node_ids.append(i)
my_list = my_pred.tolist()

In [249]:
new_list = []
for i in my_list:
    if(i == 0):
        new_list.append('ECE')
    elif(i == 2):
        new_list.append('EEE')
    if(i == 1):
        new_list.append('CSE')

In [250]:
submissions = pd.DataFrame({'id': node_ids, 'branch':new_list})
submissions = submissions.loc[nodes['test_mask']]
submissions.to_csv('submissions.csv', index=False)